# Óbitos por câncer de pulmão

Iremos analisar o número de mortes por câncer de pulmão em uma população composta por médicos britânicos, segundo sua faixa etária e o consumo médio diário de cigarros. 

Ao longo do documento realizaremos algumas análises e responderemos as seguintes questões:

- Quais são as variáveis resposta e explicativas envolvidas no estudo.
- Qual o modelo proposto.
- Ajuste do modelo e estatísticas utilizadas para isso.
- Descrição sobre a significância individual dos parâmetros. Estatísticas utilizadas para tais conclusões?
- Equação estimada pelo modelo.
- Interpretação dos parâmetros do modelo.

In [1]:
#Armazenando os dados
obitos <- c(0,3,0,3,0,1,3,3,7,29,41,45,3,16,36,11)
pop <- c(33679,21131,10599,4495,6002,4396,2813,1664,34414,25429,13271,4765,5881,6493,3466,769)
tabela <- expand.grid(FaixaEtaria = c('40-49','50-59','60-69','70-80'),Cigarros = c('0','1-9','10-30','30+'))

#Concatenando em uma tabela
dados <- cbind(tabela,obitos,pop)

#Mostrando parte dos dados
head(dados)

FaixaEtaria,Cigarros,obitos,pop
40-49,0,0,33679
50-59,0,3,21131
60-69,0,0,10599
70-80,0,3,4495
40-49,1-9,0,6002
50-59,1-9,1,4396


## Variável resposta e explicativas

Note que o objetivo deste documento é analisar o número de óbitos (`obitos`), logo, essa será a nossa variável resposta, i.e, o que desejamos estimar. A faixa etaria (`FaixaEtaria`), número de cigarros diário (`Cigarros`) e população (`pop`) serão nossas variáveis explicativas.

## Modelo estatístico

Assim, a partir das variáveis explicativas tentaremos estimar os óbitos por meio de um modelo linear generalizado (mlg ou glm em inglês) que será definido da seguinte maneira:

- Componente aleatório: `Poisson `
- Função de ligação: `Log`

A explicação da utilização desses componentes é que a distribuição `Poisson` é geralmente adequada quando a variável resposta avalia as contagens com ocorrências raras e a função de ligação `log` foi escolhida por em geral possuir bom desempenho e poder descritivo quando combinada com a `Poisson`. Deve-se lembrar também que tal ligação é conhecida por ser canônica e possui características estatísticas interessantes na modelagem.

In [2]:
#Criação do modelo utilizando glm
mod <- glm(obitos ~ offset(log(pop)) + FaixaEtaria + Cigarros, family = poisson,data=dados)
mod.sum = summary(mod)

**chamada**: `glm(formula = obitos ~ offset(log(pop)) + FaixaEtaria + Cigarros, family = poisson, data = dados)`


In [3]:
#Avaliação do modelo
cat("Residuals deviance")
t(quantile(mod.sum$deviance.resid))
cat("Coefficients")
mod.sum$coefficients

Residuals deviance

0%,25%,50%,75%,100%
-2.049188,-0.7441448,-0.0400812,0.4726137,1.216231


Coefficients

,Estimate,Std. Error,z value,Pr(>|z|)
(Intercept),-11.424072,0.5094496,-22.424343,2.278319e-111
FaixaEtaria50-59,1.769270,0.3472644,5.094878,3.489660e-07
FaixaEtaria60-69,2.897297,0.3357193,8.630120,6.128762e-18
FaixaEtaria70-80,3.791517,0.3409320,11.121035,9.911530e-29
Cigarros1-9,1.408968,0.5566426,2.531190,1.136764e-02
Cigarros10-30,2.866185,0.4182040,6.853558,7.203538e-12
Cigarros30+,3.758070,0.4269112,8.802931,1.332887e-18


**(Dispersion parameter for poisson family taken to be 1)**


- **Null deviance:** 472.596  on 15  degrees of freedom
- **Residual deviance:**  11.912  on  9  degrees of freedom
- **AIC:** 77.391
- **Number of Fisher Scoring iterations:** 5

## Ajuste do modelo

Analisando a saída da função `summary` vemos que o desvio residual é relativamente baixo e comparando-o com o desvio nulo vemos que há uma grande diferença entre eles, logo, pode-se inferir que a utilização de outras variáveis (tirando a `offset`) auxiliou na modelagem. Portanto, acreditamos que nosso modelo está bem ajustado.

## Significância dos parâmetros

Analisando o `p-value` calculado em cima do `z-value` verificamos que as suas probabilidades são bem pequenas, isso significa que a chance da hipótese nula ser aceita é muito pequena, ou seja, mesmo que utilizassemos um coeficiente de significância bem pequeno, ainda rejeitaríamos a hipótese nula. Como tal hipótese considera que os beta's são iguais a 0 e nós a rejeitamos, podemos concluir que nenhum dos coeficientes calculados devem ser desconsiderados, logo todos possuem uma alta significância no modelo.

Podemos também usar a função deviance de Poisson para verificar o ajuste do modelo, segundo o sumário, o desvio residual foi de `D = 11.912`. Comparamos este desvio com uma chi-quadrado  `x²(n-p)` com `n = 16` sendo o número de observações e `n = 4` o número de variáveis, portanto `x²(16-4) = x²(12) = 21.026`. Neste caso, `D < x²(n-p)` que implica hipótese não rejeitada.

## Equação estimada

Dado a saída da função `summary`, verificamos que a equação proposta pelo modelo possui a seguinte forma

`Y = 1.7693X1 + 2.8973X2 + 3.7915X3 + 1.4090X4 + 2.8662X5 + 3.7581X6 -11.4241`

onde

`Y` = variável resposta (número de óbitos) <br>
`X1` = Faixa etária 50-59<br>
`X2` = Faixa etária 60-69<br>
`X3` = Faixa etária 70-80<br>
`X4` = Cigarros 1-9<br>
`X5` = Cigarros 10-30<br>
`X6` = Cigarros 30+<br>


## Interpretação dos parâmetros

Os parâmetros selecionados para o modelo são "transformações" dos parâmetros originais, pois estes eram em sua maioria categóricos (tratando faixas de valor como uma categoria) e aqueles são variáveis binárias que indicam se uma categoria ocorreu ou não em determinada observação.

Não obstante os atributos pareçam ser novos, a semântica continua a mesma e o modelo faz sentido. Ao analisarmos a probabilidade de um `z-value` ser mais relevante do que aquele dado pelo modelo percebe-se que idades mais elevadas e números maiores de cigarro por dia têm uma influência maior no óbito (por isso é improvável que os pesos sejam maiores).

Importante observar que os óbitos no modelo são por câncer de pulmão e têm possivelmente alguma correlação com o consumo de cigarros.

## Conclusão

Pelas análises feitas anteriomente podemos concluir que quanto maior a idade e o número médio de cigarros, a pessoa tende a possuir uma maior
probabilidade de ter câncer de pulmão. Tal fato é coerente com a realidade, pois quanto mais avançado a idade de uma pessoa, mais chances ela tem de ter doença, assim como quanto maior o número de cigarros, maiores os danos no pulmão.